In [1]:
## Py version 3.7 
## author abdoulaye

In [9]:
from utils import *
from functions import *

### In this notebook I'll scrape data based on categoy news in Wolof.
- Each category news has multiple article
- Each article has title, text, and tag (category of news)


We'll start by pulling all links in each `category`

## 1.0 Crawler and Scraper task

In [3]:
base_url = 'https://saabal.com'
def parse_url(url):
    page = requests.get(url).text
    return bs(page)

In [4]:
# 1. Links crawler

all_links = []  
# Récup liens de categories
def get_urls(base_url):
    # category_page = requests.get(base_url).text
    # category_page_soup = bs(category_page)
    cateries_news = [a.text for a in bs(requests.get(base_url).text).find(class_="widget clearfix widget_categories").find_all("a")]
    liens_categories = [a['href'] for a in bs(requests.get(base_url).text).find(class_="widget clearfix widget_categories").find_all("a")]
    # Let's grap all link for all categories news
    for lien in liens_categories:
        page = parse_url(lien)
        for soup in page.find_all(class_="entry-title archive-entry-title mt-0"):
            link = soup.find('a')['href']
            all_links.append(link)
    return cateries_news, liens_categories, all_links

cateries_news, liens_categories, links = get_urls(base_url)

links[:10]

['https://saabal.com/ndax-yalla-moo-nu-teg-nattu-walla-sunug-nangoo-nuy-lor-mengaleel-dundug-doomi-reewum-libaan-ci-senegaal-ak-dundug-doomi-senegaal-ca-libaan/',
 'https://saabal.com/liy-firndeel-ne-yaqub-lakku-wolof-a-gen-a-ganjaru-bob-faranse/',
 'https://saabal.com/tenub-lewopool-ii-di-raykatu-nit-bu-mag-jot-na-wallam/',
 'https://saabal.com/xoolal-rekk-ni-ko-alkaati-yi-raye%f0%9f%98%a2/',
 'https://saabal.com/mbas-maa-ngi-seqi-ay-jeego-ci-afrig/',
 'https://saabal.com/metiital-gi-nuy-def-doomi-afrig-yi-ca-guwansu/',
 'https://saabal.com/lu-epp-juroom-neent-fukk-junnii-90-000i-nit-dee-nanu-ci-addina-si/',
 'https://saabal.com/askanu-faraas-noo-ngay-jure-porter-plainte-seen-njiitu-reew-ma/',
 'https://saabal.com/donaal-teremp-dal-na-ci-mbootaay-gi-yor-wer-gi-yaram-ci-addina-bi/',
 'https://saabal.com/koronaawiris-baa-ngay-jeexal-nit-nu-nuul-na-ca-amerig/']

#### Parcours de chaque catégorie, tous ces articles 

In [5]:
# 2. Data scraper

titles = []
link_to_content = []
textes = []
categories = []
data = {}
def get_data(urls):
    '''
    input: links of liens_categories
    Output: dictionary of title and link to content article converted to df
    '''
    for url in urls:
        
        page = parse_url(url)
        for soup in page.find_all(class_="entry-title archive-entry-title mt-0"):
            categorie = url.split('/')[-2]
            t = soup.text.strip()
            l = soup.find('a')['href']
            titles.append(t)
            link_to_content.append(l)
            # get text data
            content = parse_url(l)
            categories.append(categorie)
            texte = [a.text for a in content.find(class_ = "entry-content").find_all('p')]
            textes.append(texte)
            # save to dic
            data['titles'] = titles
            data['link_to_content'] = link_to_content
            data['textes'] = textes
            data['categories'] = categories
            df = pd.DataFrame(data)
    return df # A arranger

In [11]:
from utils import *
from functions import *

In [13]:
#df = get_data(liens_categories)
df.to_csv('data/origin_df.csv', encoding='utf-8', index = False)

## Prepocessing
Words tonkenizing

In [15]:
df.sample(7, random_state=1456)

,titles,link_to_content,textes,categories
15,☢️ Allaahu akbar – Yal nanu Yàlla musal,https://saabal.com/%e2%98%a2%ef%b8%8f-allaahu-...,"[Partager la publication ""☢️ Allaahu akbar – Y...",gis-gis
72,Metiital gi ñuy def doomi Afrig yi ca Guwaŋsu,https://saabal.com/metiital-gi-nuy-def-doomi-a...,[Ndaw ya toogal seen i réew ca Siin yékkati na...,politig
28,Ma tënkal la tuuti ci Saamóori Ture!,https://saabal.com/ma-tenkal-la-tuuti-ci-saamo...,"[Niki bisub tay bii 02 suwe 1900, ci la Almaam...",mboor
9,Koronaawiris baa ngay jeexal nit ñu ñuul ña ca...,https://saabal.com/koronaawiris-baa-ngay-jeexa...,"[Bunu sukkandikoo ci AFP, ca ay gox yu bari ca...",bitim-reew
5,Metiital gi ñuy def doomi Afrig yi ca Guwaŋsu,https://saabal.com/metiital-gi-nuy-def-doomi-a...,[Ndaw ya toogal seen i réew ca Siin yékkati na...,bitim-reew
86,Taskatu xibaar bii di Seex Faal Mooy indi ay ...,https://saabal.com/taskatu-xibaar-bii-di-seex-...,"[Partager la publication ""Taskatu xibaar bii ...",widewo-ndeggat
71,Saytug koppar gi jëm ci xeex koronaawiris bi –...,https://saabal.com/saytug-koppar-gi-jem-ci-xee...,[Seneraal Faraasuwaa Njaay moom lañu tabb ngir...,politig
